Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

Получившийся отчёт сохранить в формате Apache Parquet.

In [1]:
import os
import sys
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql import SparkSession

In [17]:
# Устанавливаем переменные окружения для использования текущего интерпретатора Python с PySpark
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Задаем переменную окружения для парсинга xml
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.17.0 pyspark-shell'

In [18]:
spark = SparkSession.builder.getOrCreate()
spark

In [19]:
postsData = spark.read.format('xml').option('rowTag', 'row').option("timestampFormat", 'y/M/d H:m:s').load('/content/posts_sample.xml')

In [20]:
print("Первые 5 элементов")
postsData.show(n = 5)

print("Количество элементов")
print(postsData.count())

Первые 5 элементов
+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+---+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+--------------------+--------------------+----------+
|_AcceptedAnswerId|_AnswerCount|               _Body|_ClosedDate|_CommentCount| _CommunityOwnedDate|       _CreationDate|_FavoriteCount|_Id|   _LastActivityDate|       _LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|               _Tags|              _Title|_ViewCount|
+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+---+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+--------------------+---

In [21]:
# Определяем границы временного интервала c 2010-2020
start_date = "2010-01-01"
end_date = "2020-12-31"

# Отбираем только те записи, дата создания которых попадает в указанный период
filtered_posts = postsData.where(
    (F.col("_CreationDate") >= start_date) & (F.col("_CreationDate") <= end_date)
)

filtered_posts.show(10)

+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+-----+------+----------+
|_AcceptedAnswerId|_AnswerCount|               _Body|_ClosedDate|_CommentCount| _CommunityOwnedDate|       _CreationDate|_FavoriteCount|    _Id|   _LastActivityDate|       _LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|_Tags|_Title|_ViewCount|
+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+-----+------+----------+
|             NULL|        NULL|<p>No. (And more ...|       NULL

In [22]:
languagesData = spark.read.format('csv').option('header', 'true').option("inferSchema", True).load('/content/programming-languages.csv').dropna()


In [23]:
print("Первые 5 элементов")
languagesData.show(n = 5)

print("Количество элементов")
print(languagesData.count())

Первые 5 элементов
+----------+--------------------+
|      name|       wikipedia_url|
+----------+--------------------+
|   A# .NET|https://en.wikipe...|
|A# (Axiom)|https://en.wikipe...|
|A-0 System|https://en.wikipe...|
|        A+|https://en.wikipe...|
|       A++|https://en.wikipe...|
+----------+--------------------+
only showing top 5 rows

Количество элементов
699


In [24]:
# Список названий языков
language_names = [str(x[0]) for x in languagesData.collect()]
language_names

['A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET',
 'ABSYS',
 'ACC',
 'Accent',
 'Ace DASL',
 'ACL2',
 'ACT-III',
 'Action!',
 'ActionScript',
 'Ada',
 'Adenine',
 'Agda',
 'Agilent VEE',
 'Agora',
 'AIMMS',
 'Alef',
 'ALF',
 'ALGOL 58',
 'ALGOL 60',
 'ALGOL 68',
 'ALGOL W',
 'Alice',
 'Alma-0',
 'AmbientTalk',
 'Amiga E',
 'AMOS',
 'AMPL',
 'Apex (Salesforce.com)',
 'APL',
 "App Inventor for Android's visual block language",
 'AppleScript',
 'Arc',
 'ARexx',
 'Argus',
 'AspectJ',
 'Assembly language',
 'ATS',
 'Ateji PX',
 'AutoHotkey',
 'Autocoder',
 'AutoIt',
 'AutoLISP / Visual LISP',
 'Averest',
 'AWK',
 'Axum',
 'B',
 'Babbage',
 'Bash',
 'BASIC',
 'bc',
 'BCPL',
 'BeanShell',
 'Batch (Windows/Dos)',
 'Bertrand',
 'BETA',
 'Bigwig',
 'Bistro',
 'BitC',
 'BLISS',
 'Blockly',
 'BlooP',
 'Blue',
 'Boo',
 'Boomerang',
 'Bourne shell (including',
 'bash and',
 'ksh )',
 'BREW',
 'BPEL',
 'C',
 'C--',
 'C++ – ISO/IEC 14882',
 'C# – ISO/IEC

In [25]:
# Проверяем содержит пост в тегах один из языков программирования или нет
def includes_name(x):
    tags_str = str(x._Tags).lower()
    tag = next((name for name in language_names if f'<{name.lower()}>' in tags_str), 'No')
    return (x[6], tag)

In [ ]:
# Преобразуем DataFrame в RDD и применяем функцию includes_name
posts_by_date_rdd = posts_by_date.rdd.map(includes_name)

# Фильтруем записи, где тег найден (не 'No')
filtered_rdd = posts_by_date_rdd.filter(lambda x: x[1] != 'No')

# Группируем по году и тегу, считаем количество записей в каждой группе
posts_by_date_rdd_group = (
    filtered_rdd
    .keyBy(lambda row: (row[0].year, row[1]))  # Ключ: (год, тег)
    .aggregateByKey(0, lambda count, _: count + 1, lambda a, b: a + b)  # Счётчик
    .sortBy(lambda x: x[1], ascending=False)  # Сортируем по убыванию количества
    .collect()  
)

In [27]:
# Создаём список годов в обратном порядке (2020, 2019, ..., 2010)
years_list = list(range(2010, 2021))[::-1]

df_by_years = []

# Для каждого года выбираем топ-10 языков по количеству постов
for year in years_list:
    # Фильтруем записи по году
    year_data = [row for row in posts_by_date_rdd_group if row[0][0] == year]
    df_by_years.extend(year_data[:10])

row_template = Row('Year', 'Language', 'Count')

# Создаём DataFrame из списка кортежей ((Year, Language), Count)
result_df = spark.createDataFrame(
    [row_template(year, lang, cnt) for (year, lang), cnt in df_by_years]
)

result_df.show()


+----+----------+-----+
|Year|  Language|Count|
+----+----------+-----+
|2019|    Python|  162|
|2019|JavaScript|  131|
|2019|      Java|   95|
|2019|       PHP|   59|
|2019|         R|   36|
|2019|         C|   14|
|2019|      Dart|    9|
|2019|    MATLAB|    9|
|2019|        Go|    9|
|2019|      Bash|    8|
|2018|    Python|  214|
|2018|JavaScript|  196|
|2018|      Java|  145|
|2018|       PHP|   99|
|2018|         R|   63|
|2018|         C|   24|
|2018|     Scala|   22|
|2018|TypeScript|   21|
|2018|PowerShell|   13|
|2018|      Bash|   12|
+----+----------+-----+
only showing top 20 rows



In [28]:
result_df.write.parquet("top_languages_2010_2020.parquet")